# Model construction

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import pathlib
from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from assets.functions import model_quality_assurance_preprocessing

In [2]:
# load preprocessed dataset
raw_data_preprocessed = pd.read_pickle(pathlib.Path(r'data\\processed_data\\raw_data_preprocessed.pkl'))
# Get the list of unique indices
indices = raw_data_preprocessed["measurement_data_compensated"].index.get_level_values(0).unique()
# Use groupby to split the DataFrame
grouped = raw_data_preprocessed["measurement_data_compensated"].groupby(level=0)
# get fcs data as a list of dataframes of each sample
fcs_data = [grouped.get_group(index).reset_index(drop=True) for index in indices]

# same for labels
indices = raw_data_preprocessed["reported_labels"].index.get_level_values(0).unique()
# Use groupby to split the DataFrame
grouped = raw_data_preprocessed["reported_labels"].groupby(level=0)
# get fcs data as a list of dataframes of each sample
labelset = [grouped.get_group(index).reset_index(drop=True) for index in indices]

In [3]:
# train test split and save respective indices

files_used_for_training_of_RFC = '2000 events of each file, from 0 to 50, as a single long list'

cell_subsets_of_interest = ['Lympho', 'BP', 'NKP', 'TP', 'T4P', 'T8P']

# to reduce computational costs reduce subsample the training set
train_size = 2000

x_train, x_valid, y_train, y_valid, indices_train, indices_valid = [], [], [], [], [], []
for i in range(0, 50):
    (x_train_loop, x_valid_loop, 
     y_train_loop, y_valid_loop,
    indices_train_loop, indices_valid_loop) = train_test_split(fcs_data[i], 
                                                               labelset[i][cell_subsets_of_interest],
                                                               labelset[i][cell_subsets_of_interest].index,
                                                               train_size=train_size, 
                                                               test_size=len(fcs_data[i])-train_size,
                                                               random_state=42)
    illegal_indices_train = y_train_loop.query('BP == 1 and NKP == 1').index
    illegal_indices_valid = y_valid_loop.query('BP == 1 and NKP == 1').index
    x_train.append(x_train_loop.drop(illegal_indices_train))
    y_train.append(y_train_loop.drop(illegal_indices_train))
    x_valid.append(x_valid_loop.drop(illegal_indices_valid))
    y_valid.append(y_valid_loop.drop(illegal_indices_valid))
    indices_train.append(indices_train_loop.drop(list(illegal_indices_train)))
    indices_valid.append(indices_valid_loop.drop(list(illegal_indices_valid)))

# use remaining files as testing
x_test, y_test = [], []
for i in range(50, len(fcs_data)):
    x_test_loop = fcs_data[i]
    y_test_loop = labelset[i]
    x_test.append(x_test_loop)
    y_test.append(y_test_loop)

In [4]:
# data preparation, please see model_development.parameter_optimization.ipynb
transformation = 'fluorescence channels transformed by \n None'
scaling = 'channels scaled by \n None'
domain_adaptation = 'domain adaptation \n no method applied'
best_rfc = pd.read_pickle(pathlib.Path(r'model_development\\grid_search_results\\rfc.pkl')).sort_values(by='score', ascending=False).iloc[0]
best_rfc

FileNotFoundError: [Errno 2] No such file or directory: 'model_development\\grid_search_results\\rfc.pkl'

In [15]:
best_rfc['params']

{'classifier__n_estimators': 120,
 'classifier__n_jobs': -1,
 'classifier__random_state': 42}

In [5]:
# use best performing model and parameters, please see model_development.parameter_optimization.ipynb
rfc = RandomForestClassifier(n_estimators=120, random_state=42, n_jobs=-1)
rfc.fit(pd.concat(x_train), pd.concat(y_train))

RandomForestClassifier(n_estimators=120, n_jobs=-1, random_state=42)

In [6]:
# classify all events for subsequent model quality assurance
# predictions on the training and validation set is required for uncertainty estimation

rfc_prediction = []
for i in range(100):
    prediction_loop = rfc.predict(fcs_data[i])
    prediction_loop = pd.DataFrame(prediction_loop, 
                                   columns=['pred_Lympho','pred_BP', 'pred_NKP', 'pred_TP', 
                                            'pred_T4P', 'pred_T8P'])
    rfc_prediction.append(prediction_loop)
    print(i, 'done!', end="\r")

In [7]:
# calculate cell type specific f1 scores

f1_scores = []
for i, j in zip(labelset, rfc_prediction):
    f1_scores.append(pd.DataFrame(
        f1_score(y_true=i[cell_subsets_of_interest], y_pred=j, average=None)[np.newaxis],
        columns=i[cell_subsets_of_interest].columns
        ))
f1_scores = pd.concat(f1_scores, ignore_index=True)

In [8]:
# save f1 scores

f1_scores.to_pickle(r'data/processed_data/f1_scores_model.pkl')

In [9]:
# provide the indices used for training
for i in range(50):
    indices_train.append(range(0))


# preprocess data for subsequent analyses
dataset_model_predictions =[]
for i in range(100):
    dataset_model_predictions.append(
        model_quality_assurance_preprocessing(fcs_data[i], 
                                              labelset[i],
                                              indices_train[i], 
                                              rfc_prediction[i], 
                                              i)
    )

In [10]:
# show the processed df
data_model = pd.concat(dataset_model_predictions)
data_model

measurement_data_compensated                              \
                                  FSC-A         SSC-A        FITC-A   
file event                                                            
001  0                     50573.101562   9443.280273   -192.361191   
     1                     60721.648438  15401.190430  50677.765625   
     2                     38507.070312  11470.410156    102.436935   
     3                     88148.398438  88019.195312     99.228241   
     4                     71808.062500  86550.664062    128.604401   
...                                 ...           ...           ...   
100  76983                 73832.546875  74114.460938    163.966949   
     76984                 56489.968750  44520.421875     61.773682   
     76985                 63175.050781  67636.171875     79.222885   
     76986                 56315.968750  24900.330078   5775.441406   
     76987                 74095.289062  37668.750000   2597.294189   

                                                                              \
                    PE-A     PerCP-A      PE-Cy7-A       APC-A      APC-H7-A   
file event                                                                     
001  0         49.568016  127.291069    -25.279428 -207.001297  49765.726562   
     1      11363.137695  607.533691    947.504639  -62.260956  21956.218750   
     2      13033.757812  683.279968   4808.329102 -151.480698  41288.160156   
     3        836.311340 -139.178345   2764.074951   67.033386  11105.057617   
     4       1799.095459  123.587227   5954.442383   97.762772  14570.107422   
...                  ...         ...           ...         ...           ...   
100  76983   1048.523315  309.503815  10503.620117  473.652069  11203.947266   
     76984    560.338135 -157.014450   6751.536133  240.275589   8255.482422   
     76985    752.837341  251.232635  12247.950195   54.703140  10784.350586   
     76986   8930.164062 -378.949402   1057.262939  382.759033  29675.646484   
     76987   7626.439941  -46.928322    201.112900   58.847775  27282.300781   

                                        ... reported_labels         valid_Q  \
           Pacific Blue-A     AmCyan-A  ...        TBNK Sum TP valid_Q_file   
file event                              ...                                   
001  0         107.057472  4350.436523  ...               1  1            1   
     1         706.065063   356.222687  ...               0  0            1   
     2         -24.465176    59.403057  ...               1  0            1   
     3         118.158867   203.517975  ...               0  0            1   
     4          76.168953   250.600937  ...               0  0            1   
...                   ...          ...  ...             ... ..          ...   
100  76983      73.233459   173.731506  ...               0  0            1   
     76984      60.349442    80.097862  ...               0  0            1   
     76985      21.228531   104.864441  ...               0  0            1   
     76986     811.178345   168.090225  ...               0  0            1   
     76987     811.333435    41.218067  ...               0  0            1   

                used_for_training_Q predicted_labels                           \
           used_for_training_Q_file      pred_Lympho pred_BP pred_NKP pred_TP   
file event                                                                      
001  0                            0                1       0        0       1   
     1                            0                0       0        0       0   
     2                            0                1       0        1       0   
     3                            0                0       0        0       0   
     4                            0                0       0        0       0   
...                             ...              ...     ...      ...     ...   
100  76983                        0                0       

In [11]:
data_model.to_pickle('data\\processed_data\\#1_aicontrol_dataset_pred.pkl')